In [ ]:
import pandas as pd

df = pd.read_csv('assets/data/datos.csv', delimiter=';')

#print(df.head())

print(df.columns)

Index(['Categoria', 'Enlace', 'Nombre', 'ImagenURL', 'Ingredientes',
       'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
       'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
       'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
       'Potasio(mg)'],
      dtype='object')


In [3]:

def contar_caracteres_dataset(df, columnas_excluidas):
  """
  Cuenta la cantidad total de caracteres en un DataFrame de pandas,
  excluyendo las columnas especificadas.

  Args:
    df (pd.DataFrame): El DataFrame de pandas.
    columnas_excluidas (list): Una lista de nombres de columnas para excluir.

  Returns:
    int: La cantidad total de caracteres en las columnas restantes.
  """
  
  total_caracteres = 0
  
  for columna in df.columns:
    if columna not in columnas_excluidas:
      for valor in df[columna]:
        if isinstance(valor, str): # Verifica si el valor es una cadena
          total_caracteres += len(valor)
  return total_caracteres


columnas_a_excluir = ['Enlace', 'ImagenURL']
total_caracteres = contar_caracteres_dataset(df, columnas_a_excluir)

print(f"La cantidad total de caracteres (excluyendo 'Enlace' e 'ImagenURL') es: {total_caracteres}")

La cantidad total de caracteres (excluyendo 'Enlace' e 'ImagenURL') es: 280913


In [1]:
import os
import pandas as pd
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
from sklearn.preprocessing import LabelEncoder

# Cargar variables de entorno
load_dotenv()

# Inicializar Azure OpenAI Embeddings
embeddings_model = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
    #api_version=os.getenv("api_version_openai_emb"),
    #deployment_name= os.getenv("deploy_name_emb")
)

# Cargar el dataset
df = pd.read_csv('assets/data/datos.csv', delimiter=';')  # Asegúrate de reemplazar con tu ruta

# Columnas de texto que queremos convertir
columnas_texto = ['Categoria', 'Nombre', 'Ingredientes']

# Aplicar embeddings a las columnas de texto y convertir a números
for col in columnas_texto:
    df[col + "_emb"] = df[col].astype(str).apply(lambda x: sum(embeddings_model.embed_query(x)))

# Codificar las categorías con números (Label Encoding)
label_encoder = LabelEncoder()
df["Categoria_Num"] = label_encoder.fit_transform(df["Categoria"].astype(str))

# Columnas numéricas originales + nuevas columnas transformadas
columnas_numericas = [
    'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
    'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
    'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
    'Potasio(mg)', 'Categoria_Num', 'Nombre_emb', 'Ingredientes_emb'
]

# Reemplazar NaN con 0
df[columnas_numericas] = df[columnas_numericas].fillna(0)

# Crear la nueva columna con la suma total
df["Suma_Total"] = df[columnas_numericas].sum(axis=1)

# Guardar el dataset con la nueva columna
df.to_csv("dataset_numerico.csv", index=False)

print("Dataset convertido y guardado en dataset_numerico.csv ✅")


KeyboardInterrupt: 

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd

load_dotenv()

def similitud_cosenos(vect1: list, vect2: list) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    cosine_similarity = dot_product / (norm_A * norm_B)
    return cosine_similarity


df = pd.read_csv('assets/data/datos.csv', delimiter=";") 

#respuestas = df["Ingredientes"].dropna().tolist()

respuestas = df[["Ingredientes", "Nombre", "Categoria"]].dropna().values.tolist()

embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

query = [embeddings.embed_query(text[1]) for text in respuestas]


In [5]:

query_client = "Receta sin arroz"
query_embeddings = embeddings.embed_query(query_client)


response_option = [similitud_cosenos(query_embeddings, q) for q in query]

pos = np.argmax(response_option)
request = respuestas[pos]

print(f"{query_client} - rta: {request}")

Receta sin arroz - rta: [' | 1 libra de carne molida de res magra (15% de grasa) | 1 cebolla mediana, picada | 1 chile pimiento pequeño o jalapeño, picado | 1 taza de arroz, sin cocer | 2 tazas de tomate picado, fresco o enlatado, en su jugo | 2 tazas de agua | 1 cucharada de chile en polvo (o más, a su gusto) | 1 cucharada de orégano deshidratado | 1 cucharadita de sal | 1/2 taza (2 onzas) de queso cheddar, rallado', 'Cazuela de arroz picante', 'Cena']


In [ ]:

query_client = "Receta sin arroz con 500 calorias"
query_embeddings = embeddings.embed_query(query_client)


response_option = [similitud_cosenos(query_embeddings, q) for q in query]


pos = np.argmax(response_option)
request_ingredientes, request_nombre, request_categoria = respuestas[pos]

print(f"{query_client}\nReceta: {request_nombre}\nCategoría: {request_categoria}\nIngredientes: {request_ingredientes}")

Receta sin arroz
Receta: Cazuela de arroz picante
Categoría: Cena
Ingredientes:  | 1 libra de carne molida de res magra (15% de grasa) | 1 cebolla mediana, picada | 1 chile pimiento pequeño o jalapeño, picado | 1 taza de arroz, sin cocer | 2 tazas de tomate picado, fresco o enlatado, en su jugo | 2 tazas de agua | 1 cucharada de chile en polvo (o más, a su gusto) | 1 cucharada de orégano deshidratado | 1 cucharadita de sal | 1/2 taza (2 onzas) de queso cheddar, rallado


In [4]:
from langchain_openai import AzureOpenAIEmbeddings
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd

load_dotenv()

def similitud_cosenos(vect1: list, vect2: list) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    cosine_similarity = dot_product / (norm_A * norm_B)
    return cosine_similarity

# Cargar dataset original
df = pd.read_csv('assets/data/datos.csv', delimiter=";") 

# Extraer información relevante para embeddings
columnas_relevantes = ['Categoria','Nombre', 'Ingredientes',
       'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
       'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
       'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
       'Potasio(mg)']

df_relevante = df[columnas_relevantes].dropna()

# Configurar embeddings
embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

# Crear embeddings para todas las columnas relevantes
for columna in columnas_relevantes:
    print(f"Convirtiendo columna a embeddings: {columna}")
    df_relevante[f"{columna}_Embeddings"] = df_relevante[columna].apply(lambda x: print(f"Procesando: {x}") or embeddings.embed_query(str(x)))

# Guardar nuevo dataset con embeddings
df_relevante.to_csv('assets/data/datos_embeddings.csv', index=False, sep=";")

print("Dataset con embeddings creado exitosamente.")


Convirtiendo columna a embeddings: Categoria
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando

In [9]:
import numpy as np
import pandas as pd
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

def similitud_cosenos(vect1: np.ndarray, vect2: np.ndarray) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    return dot_product / (norm_A * norm_B)

# Cargar dataset con embeddings
df_embeddings = pd.read_csv('assets/data/datos_embeddings.csv', delimiter=";")

for columna in ["Categoria_Embeddings", "Nombre_Embeddings", "Ingredientes_Embeddings", "Calorias_Embeddings"]:
    df_embeddings[columna] = df_embeddings[columna].apply(
        lambda x: np.array([float(i) for i in x.strip("[]").replace(",", "").split()])
    )


for columna in ["Categoria_Embeddings", "Nombre_Embeddings", "Ingredientes_Embeddings", "Calorias_Embeddings"]:
    df_embeddings[columna] = df_embeddings[columna].apply(
        lambda x: np.array([float(i) for i in x.strip("[]").split()]) if isinstance(x, str) else x
    )


# Configurar embeddings
embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

# Consulta del usuario
query_client = "Receta con platano"
query_embeddings = np.array(embeddings.embed_query(query_client))

# Calcular similitud con todas las filas del dataset (usando los embeddings combinados)
response_option = [
    similitud_cosenos(query_embeddings, np.mean(np.vstack([
        row["Categoria_Embeddings"], 
        row["Nombre_Embeddings"], 
        row["Ingredientes_Embeddings"], 
        row["Calorias_Embeddings"]
    ]), axis=0))  
    for _, row in df_embeddings.iterrows()
]

# Encontrar la receta más similar
pos = np.argmax(response_option)
request_ingredientes = df_embeddings.loc[pos, "Ingredientes"]
request_nombre = df_embeddings.loc[pos, "Nombre"]
request_categoria = df_embeddings.loc[pos, "Categoria"]

# Imprimir resultados
print(f"{query_client}\nReceta: {request_nombre}\nCategoría: {request_categoria}\nIngredientes: {request_ingredientes}")



Receta con platano
Receta: Galletas de plátano y avena
Categoría: Bajo en grasa
Ingredientes:  | 2 plátanos (bananas) bien maduros | 1 taza de avena (instantánea o tradicional) | 1/2 cucharadita de canela molida | 1/2 cucharadita de vainilla | 1/4 taza de pasas


In [1]:
import numpy as np
import pandas as pd
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
import json

load_dotenv()

def similitud_cosenos(vect1: np.ndarray, vect2: np.ndarray) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    return dot_product / (norm_A * norm_B)

# Cargar dataset con embeddings
df_embeddings = pd.read_csv('assets/data/datos_embeddings.csv', delimiter=";")

# Asegurarse de convertir los embeddings a arrays
for columna in ["Categoria_Embeddings", "Nombre_Embeddings", "Ingredientes_Embeddings", "Calorias_Embeddings"]:
    df_embeddings[columna] = df_embeddings[columna].apply(
        lambda x: np.array([float(i) for i in x.strip("[]").replace(",", "").split()])
    )

# Configurar embeddings
embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

# Función para filtrar recetas según criterios genericos
def filtrar_recetas(json_query, df):
    # Verificar columnas disponibles en el DataFrame
    print(f"Columnas disponibles en el DataFrame: {df.columns.tolist()}")
    
    # Iterar a través de las claves del JSON para aplicar los filtros
    for key, value in json_query.items():
        if value:  # Solo aplicar el filtro si el valor no es vacío o None
            if key == "calorías":
                # Filtrar por calorías
                calorias_deseadas_value = float(value.replace("kcal", "").strip())
                df = df[df["Calorias"].apply(lambda x: float(str(x).replace("kcal", "").strip()) <= calorias_deseadas_value)]
            
            elif key == "ingredientes_deseados":
                # Filtrar por ingredientes deseados
                df = df[df["Ingredientes"].apply(lambda x: all(ingrediente in x.lower() for ingrediente in value))]
            
            elif key == "ingredientes_no_deseados":
                # Filtrar por ingredientes no deseados
                df = df[df["Ingredientes"].apply(lambda x: all(ingrediente not in x.lower() for ingrediente in value))]
            
            else:
                # Filtros generales para cualquier otra columna
                if key in df.columns:  # Comprobar si la columna existe
                    df = df[df[key].apply(lambda x: any(val in str(x).lower() for val in value))]
                else:
                    print(f"Advertencia: La columna '{key}' no se encuentra en el DataFrame.")
    
    return df

# Ejemplo de query en formato JSON con criterios flexibles
query_client = {
    "calorías": "500kcal",
    "ingredientes_deseados": ["arroz"],
    "ingredientes_no_deseados": ["carne"]
}

# Filtrar recetas usando los criterios del JSON
df_filtrado = filtrar_recetas(query_client, df_embeddings)

# Convertir el diccionario a una cadena de texto
query_text = f"Calorías: {query_client.get('calorías', '')}, Ingredientes deseados: {', '.join(query_client.get('ingredientes_deseados', []))}, Ingredientes no deseados: {', '.join(query_client.get('ingredientes_no_deseados', []))}"

# Pasar la cadena a la función de embeddings
query_embeddings = np.array(embeddings.embed_query(query_text))

# Calcular similitud con todas las filas del dataset filtrado
response_option = [
    similitud_cosenos(query_embeddings, np.mean(np.vstack([
        row["Categoria_Embeddings"], 
        row["Nombre_Embeddings"], 
        row["Ingredientes_Embeddings"], 
        row["Calorias_Embeddings"]
    ]), axis=0))  
    for _, row in df_filtrado.iterrows()
]


# Verificar si el DataFrame filtrado tiene al menos una fila
if len(df_filtrado) > 0:
    # Restablecer el índice para garantizar que la posición sea válida
    df_filtrado_reset = df_filtrado.reset_index(drop=True)
    
    # Calcular similitudes
    response_option = [
        similitud_cosenos(query_embeddings, np.mean(np.vstack([
            row["Categoria_Embeddings"], 
            row["Nombre_Embeddings"], 
            row["Ingredientes_Embeddings"], 
            row["Calorias_Embeddings"]
        ]), axis=0))  
        for _, row in df_filtrado_reset.iterrows()
    ]
    
    # Verificar si la lista de similitudes tiene elementos
    if len(response_option) > 0:
        pos = np.argmax(response_option)  # Obtener la posición del máximo valor
        request_ingredientes = df_filtrado_reset.loc[pos, "Ingredientes"]
        request_nombre = df_filtrado_reset.loc[pos, "Nombre"]
        request_categoria = df_filtrado_reset.loc[pos, "Categoria"]
        
        # Imprimir resultados
        print(f"{query_client}\nReceta: {request_nombre}\nCategoría: {request_categoria}\nIngredientes: {request_ingredientes}")
    else:
        print("No se encontraron opciones de respuesta.")
else:
    print("No se encontraron recetas que coincidan con los criterios.")


Columnas disponibles en el DataFrame: ['Categoria', 'Nombre', 'Ingredientes', 'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)', 'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)', 'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)', 'Potasio(mg)', 'Categoria_Embeddings', 'Nombre_Embeddings', 'Ingredientes_Embeddings', 'Calorias_Embeddings', 'Grasa Total(gr)_Embeddings', 'Colesterol(mg)_Embeddings', 'Sodio(mg)_Embeddings', 'Carbohidrato Total(gr)_Embeddings', 'Proteina(gr)_Embeddings', 'Vitamina A(mcg)_Embeddings', 'Vitamina C(mg)_Embeddings', 'Vitamina D(mcg)_Embeddings', 'Calcio(mg)_Embeddings', 'Hierro(mg)_Embeddings', 'Potasio(mg)_Embeddings']
{'calorías': '500kcal', 'ingredientes_deseados': ['arroz'], 'ingredientes_no_deseados': ['carne']}
Receta: Arroz con pollo y verduras
Categoría: Bajo en grasa
Ingredientes:  | 1 cucharada de aceite vegetal | 1 taza de cebolla picada | 1 libra de pechuga de pollo deshuesada y sin piel, cortada en pedacitos | 3 d

In [5]:
import pandas as pd

# Cargar los datasets
datos = pd.read_csv("assets/data/datos.csv", delimiter=";")
datos_embeddings = pd.read_csv('assets/data/datos_embeddings.csv', delimiter=";")

# Verificar que las columnas existen en 'datos'
columnas_a_copiar = ["Enlace", "ImagenURL"]
for col in columnas_a_copiar:
    if col not in datos.columns:
        raise KeyError(f"La columna '{col}' no está en datos.csv")

# Agregar las columnas de 'datos' a 'datos_embeddings'
datos_embeddings = datos_embeddings.join(datos[columnas_a_copiar])

# Guardar el nuevo dataset
datos_embeddings.to_csv("datos_embeddings_actualizado.csv", index=False)

print("Las columnas 'Enlace' e 'ImagenURL' fueron agregadas exitosamente a datos_embeddings.csv")


KeyboardInterrupt: 

In [7]:
import pandas as pd

datos_embeddings = pd.read_csv('assets/data/datos_embeddings_actualizado.csv')

print(datos_embeddings.columns)


Index(['Categoria', 'Nombre', 'Ingredientes', 'Calorias', 'Grasa Total(gr)',
       'Colesterol(mg)', 'Sodio(mg)', 'Carbohidrato Total(gr)', 'Proteina(gr)',
       'Vitamina A(mcg)', 'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)',
       'Hierro(mg)', 'Potasio(mg)', 'Categoria_Embeddings',
       'Nombre_Embeddings', 'Ingredientes_Embeddings', 'Calorias_Embeddings',
       'Grasa Total(gr)_Embeddings', 'Colesterol(mg)_Embeddings',
       'Sodio(mg)_Embeddings', 'Carbohidrato Total(gr)_Embeddings',
       'Proteina(gr)_Embeddings', 'Vitamina A(mcg)_Embeddings',
       'Vitamina C(mg)_Embeddings', 'Vitamina D(mcg)_Embeddings',
       'Calcio(mg)_Embeddings', 'Hierro(mg)_Embeddings',
       'Potasio(mg)_Embeddings', 'Enlace', 'ImagenURL'],
      dtype='object')


In [ ]:
import numpy as np
import pandas as pd
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
import json

load_dotenv()

def similitud_cosenos(vect1: np.ndarray, vect2: np.ndarray) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    return dot_product / (norm_A * norm_B)

# Cargar dataset con embeddings
df_embeddings = pd.read_csv('assets/data/datos_embeddings_actualizado.csv')

# Convertir los embeddings a arrays
for columna in [
    "Categoria_Embeddings", "Nombre_Embeddings", "Ingredientes_Embeddings", "Calorias_Embeddings",
    "Grasa Total(gr)_Embeddings", "Colesterol(mg)_Embeddings", "Sodio(mg)_Embeddings", "Carbohidrato Total(gr)_Embeddings",
    "Proteina(gr)_Embeddings", "Vitamina A(mcg)_Embeddings", "Vitamina C(mg)_Embeddings", "Vitamina D(mcg)_Embeddings",
    "Calcio(mg)_Embeddings", "Hierro(mg)_Embeddings", "Potasio(mg)_Embeddings"
]:
    df_embeddings[columna] = df_embeddings[columna].apply(
        lambda x: np.array([float(i) for i in x.strip("[]").replace(",", "").split()])
    )

# Configurar embeddings
embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

# Función para filtrar recetas según criterios
def filtrar_recetas(json_query, df):
    for key, value in json_query.items():
        if value is not None:
            if key == "calorías":
                df = df[df["Calorias"] <= value]
            elif key == "proteínas":
                df = df[df["Proteina(gr)"] >= value]
            elif key == "carbohidratos":
                df = df[df["Carbohidrato Total(gr)"] <= value]
            elif key == "grasas":
                df = df[df["Grasa Total(gr)"] <= value]
            elif key == "ingredientes_deseados":
                df = df[df["Ingredientes"].apply(lambda x: all(ing.lower() in x.lower() for ing in value))]
            elif key == "ingredientes_no_deseados":
                df = df[df["Ingredientes"].apply(lambda x: all(ing.lower() not in x.lower() for ing in value))]
            elif key == "vitaminas_minerales":
                for vit in value:
                    if vit in df.columns:
                        df = df[df[vit] > 0]  # Filtra solo si el valor de la vitamina/mineral es mayor a 0
    return df

# Query de ejemplo
query_client = {
    "calorías": None,
    "proteínas": None,
    "carbohidratos": None,
    "grasas": None,
    "ingredientes_deseados": ["Arroz", "Carne"],
    "ingredientes_no_deseados": ["zanahoria"],
    "vitaminas_minerales": ["Vitamina C(mg)"]
}

df_filtrado = filtrar_recetas(query_client, df_embeddings)

query_text = f"Ingredientes deseados: {', '.join(query_client.get('ingredientes_deseados', []))}, " \
              f"Ingredientes no deseados: {', '.join(query_client.get('ingredientes_no_deseados', []))}, " \
              f"Vitaminas y minerales: {', '.join(query_client.get('vitaminas_minerales', []))}"

query_embeddings = np.array(embeddings.embed_query(query_text))

if not df_filtrado.empty:
    df_filtrado_reset = df_filtrado.reset_index(drop=True)
    response_option = [
        similitud_cosenos(query_embeddings, np.mean(np.vstack([
            row["Categoria_Embeddings"], row["Nombre_Embeddings"], row["Ingredientes_Embeddings"],
            row["Calorias_Embeddings"], row["Grasa Total(gr)_Embeddings"], row["Colesterol(mg)_Embeddings"],
            row["Sodio(mg)_Embeddings"], row["Carbohidrato Total(gr)_Embeddings"], row["Proteina(gr)_Embeddings"],
            row["Vitamina A(mcg)_Embeddings"], row["Vitamina C(mg)_Embeddings"], row["Vitamina D(mcg)_Embeddings"],
            row["Calcio(mg)_Embeddings"], row["Hierro(mg)_Embeddings"], row["Potasio(mg)_Embeddings"]
        ]), axis=0))
        for _, row in df_filtrado_reset.iterrows()
    ]
    
    if response_option:
        pos = np.argmax(response_option)
        request_data = df_filtrado_reset.loc[pos]
        
        respuesta_formateada = {
            "categoria": str(request_data["Categoria"]),
            "enlace": str(request_data["Enlace"]),
            "nombre": str(request_data["Nombre"]),
            "imagenURL": str(request_data["ImagenURL"]),
            "ingredientes": str(request_data["Ingredientes"]),
            "calorias": int(request_data["Calorias"]),
            "grasa": float(request_data["Grasa Total(gr)"]),
            "colesterol": int(request_data["Colesterol(mg)"]),
            "sodio": int(request_data["Sodio(mg)"]),
            "carbohidratos": float(request_data["Carbohidrato Total(gr)"]),
            "proteina": float(request_data["Proteina(gr)"]),
            "vitaminaA": float(request_data["Vitamina A(mcg)"]),
            "vitaminaC": float(request_data["Vitamina C(mg)"]),
            "vitaminaD": float(request_data["Vitamina D(mcg)"]),
            "calcio": float(request_data["Calcio(mg)"]),
            "hierro": float(request_data["Hierro(mg)"]),
            "potasio": float(request_data["Potasio(mg)"]),
        }
        print(json.dumps(respuesta_formateada, indent=4, ensure_ascii=False))
    else:
        print("No se encontraron opciones de respuesta.")
else:
    print("No se encontraron recetas que coincidan con los criterios.")


{
    "categoria": "Sin lácteos",
    "enlace": "https://medlineplus.gov/spanish/recetas/vegetales-con-carne-a-la-sarten-al-estilo-mexicano/",
    "nombre": "Pollo con piña y verduras",
    "imagenURL": "https://medlineplus.gov/images/recipe_mexicanvegetableandbeefskilletmeal.jpg",
    "ingredientes": " | 2 cucharaditas de aceite vegetal | 3 pechugas de pollo, sin hueso, sin piel y cortadas en trozos de 2 pulgadas | 2 tajadas de raíz de jengibre fresco, pelado y picado o 1/2 cucharadita de jengibre en polvo | 1 diente de ajo picados muy fino o 1/4 de cucharadita de ajo en polvo | 1 lata (8 onzas) piña en trozos, escurrida (guarde el jugo) | 1/2 taza de caldo de pollo o agua* | 2 cucharadas de vinagre | 4 tazas de verduras en rebanadas, como apio, pimiento morrón, cebolla y champiñones | 1/2 taza de jugo de naranja | 2 cucharadas de salsa de soya | 1 cucharada de azúcar | 3 cucharadas de maicena | 1 tomate mediano, cortado en pedazos pequeños",
    "calorias": 100,
    "grasa": 2.0,
   

In [1]:
print(df_embeddings.columns)


NameError: name 'df_embeddings' is not defined